In [3]:
import pandas as pd
import numpy as np
from nilearn.connectome import sym_matrix_to_vec, vec_to_sym_matrix
from nilearn.plotting import plot_matrix
import matplotlib.pyplot as plt

from scipy import stats

data = np.load("/home/finc/Dropbox/Projects/LearningBrain/data/neuroimaging/LearningBrain_matrices_conn_style.npy")
dualnback = data[:,:,1:3,:,:]

dualnback.shape

x = dualnback[0,0,0,:,:]
x.shape

#np.save('/home/finc/Dropbox/Projects/LearningBrain/data/neuroimaging/test_matrix', x)


(264, 264)

In [4]:
# Selecting subjects which finished the study

groups = pd.read_csv('/home/finc/Dropbox/Projects/LearningBrain/github/LearningBrain_networks/data/behavioral/group_assignment.csv')

trained = (groups.group == 'Experimental') | (groups.group == 'Control')
trained_subs = groups[trained]['sub']
subs = pd.Series.tolist(trained_subs)
subs[:5]

['sub-01', 'sub-02', 'sub-04', 'sub-05', 'sub-06']

In [5]:
# Selecting subjects to exclude

# dualnback
dualnback_exclude = ['sub-13', 'sub-21', 'sub-23', 'sub-50'] # higly motion subjects in one of four sessions
rest_exclude = ['sub-20', 'sub-44', 'sub-21', 'sub-46', 'sub-47'] # higly motion subjects in one of four sessions / missing data(20-44)

dual_vector = []
rest_vector = []
first_vector = np.array(subs) ==  'sub-21' # subject with highly motion on first session

dual_vector = [False if sub in dualnback_exclude else True for sub in subs]
rest_vector = [False if sub in rest_exclude else True for sub in subs]


In [6]:
# Creating groups vectors

groups = pd.read_csv('/home/finc/Dropbox/Projects/LearningBrain/github/LearningBrain_networks/data/behavioral/group_assignment.csv')

trained = (groups.group == 'Experimental') | (groups.group == 'Control')
trained_subs = groups[trained]

experimental = (trained_subs == 'Experimental')
control = (trained_subs == 'Control')

exp_vector = experimental['group'].values * dual_vector
con_vector = control['group'].values * dual_vector

print(sum(exp_vector))
print(sum(con_vector))

21
21


In [7]:
modules = pd.read_csv('/home/finc/Dropbox/Projects/LearningBrain/code/modules.txt', sep = " ", header = None)

mod = modules[0].values

DMFP = (mod == 'DM')|(mod == 'FP')
DM = (mod == 'DM')
FP = (mod == 'FP')

DMFP.shape

(264,)

In [9]:
#dualnback = dualnback[:,:,:,:,DMFP]
#dualnback = dualnback[:,:,:,DMFP,:]

In [11]:

exp_dual_2back_1 = sym_matrix_to_vec(dualnback[:,3,0,:,:], discard_diagonal = True)
exp_dual_2back_2 = sym_matrix_to_vec(dualnback[:,3,1,:,:], discard_diagonal = True)




In [12]:
stat, pvalues = stats.ttest_rel(exp_dual_2back_1 , exp_dual_2back_2)

In [13]:
import statsmodels.stats.multitest as ssm

_, pvals_corrected, _, _ = ssm.multipletests(pvalues, alpha = 0.05, method = 'fdr_bh')
pvals_corrected_thr = np.zeros((len(pvals_corrected)))

pvals = np.array([0 if p >= 0.05 else 1 for p in pvals_corrected])

In [14]:
sum(pvals)

sum(pvals)

0

In [99]:
pvals_corrected

array([ nan])

In [16]:
wei_vector = stat * pvals 

In [17]:
diag = np.zeros((264))
matrix = vec_to_sym_matrix(wei_vector, diagonal = diag)
#matrix = vec_to_sym_matrix(pvals_corrected)


matrix_bin = vec_to_sym_matrix(pvals, diagonal = diag)

ValueError: Vector of unsuitable shape (54780,) can not be transformed to a symmetric matrix.

In [ ]:
matrix.shape

In [ ]:
plot_matrix(matrix)

In [ ]:
thr_matrix = ((matrix >= 4.5) | (matrix <= -4.5)) * matrix

thr_matrix.shape

In [ ]:
from nilearn import datasets # for fetching atlas
from nilearn import plotting

# Loading Power ROIs coordinates

power = datasets.fetch_coords_power_2011()
power_coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

In [ ]:
plotting.plot_connectome(thr_matrix, power_coords)#edge_vmin = -6, edge_vmax = 6)